In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import os
from VAERSFileDownloader import updateVAERSFiles
from datetime import datetime
from DateProvider import DateProvider
from InternationalVaersCovid19Provider import getInternationalVaersCovid19,get_international_VAERSVAX_VAERSSYMPTOMS_Covid19
from BatchCodeTableHtmlUpdater import updateBatchCodeTableHtmlFile
from BatchCodeTablePersister import createAndSaveBatchCodeTables

In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
years_from_2020_to_present = list(range(2020, datetime.now().year + 1))

In [ ]:
updateVAERSFiles(
        years = years_from_2020_to_present,
        workingDirectory = os.getcwd())

In [ ]:
international_VAERSVAX_Covid19, international_VAERSSYMPTOMS = get_international_VAERSVAX_VAERSSYMPTOMS_Covid19(years = years_from_2020_to_present)

In [ ]:
international_VAERSVAX_Covid19

In [ ]:
international_VAERSSYMPTOMS

In [ ]:
from SymptomByBatchcodeTableFactory import SymptomByBatchcodeTableFactory

symptomByBatchcodeTable = SymptomByBatchcodeTableFactory.createSymptomByBatchcodeTable(international_VAERSVAX_Covid19, international_VAERSSYMPTOMS)
symptomByBatchcodeTable

In [ ]:
isGlobal = True

In [ ]:
from SymptomHistogramByBatchcodeTableFactory import SymptomHistogramByBatchcodeTableFactory

symptomHistogramByBatchcodeTable = SymptomHistogramByBatchcodeTableFactory.createGlobalSymptomHistogramByBatchcodeTable(symptomByBatchcodeTable) if isGlobal else SymptomHistogramByBatchcodeTableFactory.createSymptomHistogramByBatchcodeTable(symptomByBatchcodeTable)
symptomHistogramByBatchcodeTable

In [ ]:
from HistogramTable2DictTableConverter import HistogramTable2DictTableConverter

dictByBatchcodeTable = HistogramTable2DictTableConverter.convertGlobalHistogramTable2DictTable(symptomHistogramByBatchcodeTable) if isGlobal else HistogramTable2DictTableConverter.convertHistogramTable2DictTable(symptomHistogramByBatchcodeTable)
dictByBatchcodeTable

In [ ]:
from HtmlUtils import getCountries
from MultiIndexValuesProvider import MultiIndexValuesProvider
from HistogramDescriptionPersister import HistogramDescriptionPersister
import shutil

if isGlobal:
    batchcodes = MultiIndexValuesProvider.getValues(dictByBatchcodeTable.index)
    batchcodes = {batchcode for batchcode in batchcodes if batchcode != 'nan'}
    # directory = f'../docs/data/histograms/global'
    directory = f'../docs/data/histograms'
    shutil.rmtree(directory, ignore_errors = True)
    histogramDescriptionPersister = HistogramDescriptionPersister(directory)
    histogramDescriptionPersister.saveHistogramDescriptionsForBatchcodes(
        batchcodes,
        dictByBatchcodeTable,
        progress = lambda count, size, batchcode: print(f'{count}/{size}: {batchcode}'))
else:
    countries = getCountries(international_VAERSVAX_Covid19)
    for count, country in enumerate(countries, start = 1):
        # FK-TODO: use https://github.com/tqdm/tqdm
        print(f'saving histograms for country {count}/{len(countries)}: {country}')
        dictByBatchcodeTable4Country = dictByBatchcodeTable[dictByBatchcodeTable['COUNTRY'] == country]
        batchcodes = MultiIndexValuesProvider.getValues(dictByBatchcodeTable4Country.index)
        batchcodes = {batchcode for batchcode in batchcodes if batchcode != 'nan'}
        directory = f'../docs/data/histograms/{country}'
        shutil.rmtree(directory, ignore_errors = True)
        histogramDescriptionPersister = HistogramDescriptionPersister(directory)
        histogramDescriptionPersister.saveHistogramDescriptionsForBatchcodes(
            batchcodes,
            dictByBatchcodeTable4Country,
            progress = lambda count, size, batchcode: print(f'{count}/{size}: {batchcode}'))

In [ ]:
# batchcodes = international_VAERSVAX_Covid19['VAX_LOT'].dropna().drop_duplicates().to_list()
# batchcodes

In [ ]:
internationalVaersCovid19 = getInternationalVaersCovid19(years = years_from_2020_to_present)
internationalVaersCovid19

In [ ]:
updateBatchCodeTableHtmlFile(internationalVaersCovid19, batchCodeTableHtmlFile="../docs/batchCodeTable.html")

In [ ]:
createAndSaveBatchCodeTables(
    internationalVaersCovid19,
    minADRsForLethality = 100,
    onCountryProcessed = display)